In [3]:
import numpy as np
import matplotlib.pylab as plt
import cv2 as cv

def finding_contours(img1, img2):
    difference_in_images = cv.absdiff(img1, img2)
    grayed_image = cv.cvtColor(difference_in_images, cv.COLOR_BGR2GRAY)
    image_in_blur = cv.GaussianBlur(grayed_image, (5, 5), 0)
    _, threshold_for_image = cv.threshold(image_in_blur, 20, 255, cv.THRESH_BINARY)
    dilated_img = cv.dilate(threshold_for_image, None, iterations=3)
    contours_for_image, _ = cv.findContours(dilated_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    return contours_for_image


def tracking_object(frame, vertices):
    mask = np.zeros_like(frame)
    channel_count = frame.shape[2]
    match_mask_color = (255,) * channel_count
    cv.fillPoly(mask, vertices, match_mask_color)
    masked_img = cv.bitwise_and(frame, mask)
    return masked_img
def draw_the_lines(img1, lines):
    img1 = np.copy(img1)
    blank_img = np.zeros((img1.shape[0], img1.shape[1], 3), dtype=np.uint8)

    for line in lines:
        global x1, x2, y1, y2
        for (x1, y1, x2, y2) in line:
            cv.line(blank_img, (x1, y1), (x2, y2), (255, 255, 255), thickness=5)
            cv.putText(img1, "ROLLING_ON", (50, 30), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    img1 = cv.addWeighted(img1, 0.8, blank_img, 1, 0.0)
    return img1
def region_of_interest(img1, vertices):
    mask = np.zeros_like(img1)
    #channel_count = img1.shape[2]
    #match_mask_color = (255,) * channel_count
    match_mask_color = 255
    cv.fillPoly(mask, vertices, match_mask_color)
    masked_img = cv.bitwise_and(img1, mask)
    cv.imshow('masked_img', masked_img)
    return masked_img
#img = cv2.imread('runoutpic.png')
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def process(img):
    #print(img.shape)
    height = img.shape[0]
    width = img.shape[1]
    #print(height)
    #print(width)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    canny_img = cv.Canny(gray, 100, 200)
    cropped_img = region_of_interest(canny_img, np.array([roi_of_crease], np.int32))
    lines = cv.HoughLinesP(cropped_img, rho=5, theta=np.pi / 60, threshold=160, lines=np.array([]), minLineLength=40,
                            maxLineGap=25)
    drawn_img = draw_the_lines(img, lines)
    return drawn_img


cap =cv.VideoCapture('notout3.mp4')
count = 0
x = 0
ret, frame = cap.read()
ret, bat1 = cap.read()
ret, bat2 = cap.read()
ret, stump1 = cap.read()
ret, stump2 = cap.read()
cv.putText(stump2, "SELECT_THE_STUMP :", (50, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
roi_s = cv.selectROI(stump2)
cv.putText(frame, "SELECT_THE_CREASE :", (50, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
roi_c = cv.selectROI(frame)
x1 = roi_c[0]
y1 = roi_c[1] + roi_c[3]
x2 = roi_c[0]
y2 = roi_c[1]
x3 = roi_c[0] + roi_c[2]
y3 = roi_c[1]
x4 = roi_c[0] + roi_c[2]
y4 = roi_c[1] + roi_c[3]
roi_of_crease = [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]

while(cap.isOpened()):
    cv.putText(frame, "STATUS_OF_THE_WICKET :", (50, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
    frame = process(frame)
    # INFO_OF_BAT_FRAME
    height_of_bat1 = bat1.shape[0]
    height_of_bat2 = bat2.shape[0]

    width_of_bat1 = bat1.shape[1]
    width_of_bat2 = bat2.shape[1]

    # INFO_OF_STUMP_FRAME
    height_of_stump1 = stump1.shape[0]
    height_of_stump2 = stump2.shape[0]

    width_of_stump1 = stump1.shape[1]
    width_of_stump2 = stump2.shape[1]

    #  ROI __ FOR __ THE __ STUMP
    a1 = roi_s[0]
    b1 = roi_s[1] + roi_s[3]
    a2 = roi_s[0]
    b2 = roi_s[1]
    a3 = roi_s[0] + roi_s[2]
    b3 = roi_s[1]
    a4 = roi_s[0] + roi_s[2]
    b4 = roi_s[1] + roi_s[3]

    roi_of_stump = [(a1, b1),(a2, b2), (a3, b3), (a4, b4)]

    # CROPPED_IMAGE_FOR_BAT
    bat_img1 = tracking_object(bat1, np.array([roi_of_crease], np.int32), )
    bat_img2 = tracking_object(bat2, np.array([roi_of_crease], np.int32), )

    # CROPPED_IMAGE_FOR_STUMP
    stump_img1 = tracking_object(stump1, np.array([roi_of_stump], np.int32), )
    stump_img2 = tracking_object(stump2, np.array([roi_of_stump], np.int32), )

    # PROCESSING_BAT_IMAGES
    bat_in_gray_1 = cv.cvtColor(bat_img1, cv.COLOR_BGR2GRAY)
    canny_img_for_bat_1 = cv.Canny(bat_in_gray_1, 100, 200)
    bat_in_gray_2 = cv.cvtColor(bat_img2, cv.COLOR_BGR2GRAY)
    canny_img_for_bat_2 = cv.Canny(bat_in_gray_2, 100, 200)

    # PROCESSING_STUMP_IMAGES
    stump_in_gray_1 = cv.cvtColor(stump_img1, cv.COLOR_BGR2GRAY)
    canny_img_for_stump_1 = cv.Canny(stump_in_gray_1, 100, 200)
    stump_in_gray_2 = cv.cvtColor(stump_img2, cv.COLOR_BGR2GRAY)
    canny_img_for_stump_2 = cv.Canny(stump_in_gray_2, 100, 200)

    # FINDING_CONTOURS_FOR_BAT
    contour_for_the_bat = finding_contours(bat_img1, bat_img2)
    for contour_bat in contour_for_the_bat:
        if cv.contourArea(contour_bat) < 250:
            continue
        (x, y, w, h) = cv.boundingRect(contour_bat)
        cv.drawContours(bat_img1, contour_for_the_bat, -1, (0, 255, 0), 5)
        cv.imshow('bat', bat_img1)
        cv.drawContours(frame, contour_for_the_bat, -1, (0, 255, 0), 5)
        if x1 >= x:
            if count == 2:
                break
            else:
                count = 1
                break
    bat1 = bat_img2
    # FINDING_CONTOURS_FOR_STUMP
    contour_for_the_stump = finding_contours(stump_img1, stump_img2)
    stump1 = stump_img2
    for contour_stump in contour_for_the_stump:
        if cv.contourArea(contour_stump) < 100:
            continue
        (a, b, c, d) = cv.boundingRect(contour_stump)
        cv.drawContours(stump_img1, contour_for_the_stump, -1, (0, 0, 255), 5)
        cv.imshow('stump_img', stump_img1)
        cv.drawContours(frame, contour_for_the_stump, -1, (0, 0, 255), 5)
        if (a):
            print('WHEN THE STUMP HITS :')
            if x!=0:
              print('CREASE = ' + str(x1) + ' BAT = ' + str(x))
            else:
                print('BAT DOES NOT REACH NEAR THE CREASE')
            if count == 1:
                break
            else:
                count = 2
                break

    if count == 1:
        cv.putText(frame, "NOT_OUT", (300, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv.putText(frame, "REASON : ", (50, 90), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv.putText(frame, "BAT_REACHES_THE_CREASE_FIRST", (300, 90), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    if count == 2:
        cv.putText(frame, "OUT", (300, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        cv.putText(frame, "REASON : ", (50, 90), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv.putText(frame, "BALL REACHES THE STUMP FIRST", (300, 90), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv.imshow('DECISION_PENDING', frame)
    ret, bat2 = cap.read()
    ret, stump2 = cap.read()
    ret, frame = cap.read()

    if cv.waitKey(500) & 0xFF == ord('q'):
         break
cap.release()
cv.destroyAllWindows()